In [1]:
from accutuning_client.client import Client

In [2]:
client = Client('localhost', 8000)

In [3]:
client.login('autoinsight', 'autoinsight')

True

In [4]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 5개 있습니다.


5 Experiments
Experiment(id=8, name=Experiment-8, dataset.name=iris1_52cCr52, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=28)
Experiment(id=4, name=Experiment-4, dataset.name=iris1_hqYy0kH, dataset.colCount=5, status=ready, estimatorType=CLASSIFIER, metric=ACCURACY)
Experiment(id=3, name=Experiment-3, dataset.name=iris1_D5MRffN, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=59, deploymentsCnt=2)
Experiment(id=2, name=Experiment-2, dataset.name=iris1, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=28)
Experiment(id=1, name=Experiment-1, dataset.name=diabetes2, dataset.colCount=11, status=finished, estimatorType=REGRESSOR, metric=NEG_MEAN_SQUARED_ERROR, modelsCnt=52, deploymentsCnt=2)

In [5]:
# Local File에서 Experiment를 생성한다. 
experiment_new = client.create_experiment_from_file('/Users/ahaljh/Downloads/iris1.csv')
experiment_new

Experiment(id=9, name=Experiment-9, status=creating, estimatorType=classifier, metric=accuracy)

In [6]:
experiment_new

Experiment(id=9, name=Experiment-9, dataset.name=iris1_13FrbNs, dataset.colCount=5, status=ready, estimatorType=CLASSIFIER, metric=ACCURACY)

In [7]:
# 전처리를 추천받아 preprocessor config를 변경 
experiment_new.preprocessor_config_recommend()

In [8]:
# 지정한 config 설정대로 전처리를 실시
experiment_new.preprocess()

In [9]:
# Run AutoML
experiment_new.run()

True

In [10]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 6개 있습니다.


6 Experiments
Experiment(id=9, name=Experiment-9, dataset.name=iris1_13FrbNs, dataset.colCount=5, status=learning, estimatorType=CLASSIFIER, metric=ACCURACY)
Experiment(id=8, name=Experiment-8, dataset.name=iris1_52cCr52, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=28)
Experiment(id=4, name=Experiment-4, dataset.name=iris1_hqYy0kH, dataset.colCount=5, status=ready, estimatorType=CLASSIFIER, metric=ACCURACY)
Experiment(id=3, name=Experiment-3, dataset.name=iris1_D5MRffN, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=59, deploymentsCnt=2)
Experiment(id=2, name=Experiment-2, dataset.name=iris1, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=28)
Experiment(id=1, name=Experiment-1, dataset.name=diabetes2, dataset.colCount=11, status=finished, estimatorType=REGRESSOR, metric=NEG_MEAN_SQUARED_ERROR, modelsCnt=52, deploymentsCnt=2)

In [11]:
# 시간이 오래 걸리니까 완료된 experiment를 다시 선택
experiment = experiments.get(id=3)
# experiment = experiments[1]  # 이 형태도 가능함
experiment

Experiment(id=3, name=Experiment-3, dataset.name=iris1_D5MRffN, dataset.colCount=5, status=finished, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=59, deploymentsCnt=2)

In [12]:
# Leaderboard 정보 구해오기 
leaderboard = experiment.leaderboard()
print(f'leaderboad의 model 갯수는 {len(leaderboard)}')
leaderboard

leaderboad의 model 갯수는 59


[{'id': '83',
  'score': 1.0,
  'trainScore': 0.9619047619047619,
  'validScore': 1.0,
  'testScore': 0.0,
  'estimatorName': 'xgboost',
  'generator': 'optuna-pycaret',
  'file': {'size': '133448', 'sizeHumanized': '133.4 kB'},
  'deployedStatus': 'DONE'},
 {'id': '86',
  'score': 1.0,
  'trainScore': 0.9619047619047619,
  'validScore': 1.0,
  'testScore': 0.0,
  'estimatorName': 'logistic_regression',
  'generator': 'optuna-pycaret',
  'file': {'size': '2380', 'sizeHumanized': '2.4 kB'},
  'deployedStatus': 'DONE'},
 {'id': '87',
  'score': 1.0,
  'trainScore': 0.9619047619047619,
  'validScore': 1.0,
  'testScore': 0.0,
  'estimatorName': 'logistic_regression',
  'generator': 'optuna-pycaret',
  'file': {'size': '1533', 'sizeHumanized': '1.5 kB'},
  'deployedStatus': None},
 {'id': '89',
  'score': 1.0,
  'trainScore': 0.9619047619047619,
  'validScore': 1.0,
  'testScore': 0.0,
  'estimatorName': 'random_forest',
  'generator': 'optuna-pycaret',
  'file': {'size': '126661', 'sizeHu

In [13]:
# 가장 순위 높은 모델 선택
model = leaderboard.best_model()
# model = leaderboard[0]  # 이 형태도 가능함
model

{'id': '83',
 'score': 1.0,
 'trainScore': 0.9619047619047619,
 'validScore': 1.0,
 'testScore': 0.0,
 'estimatorName': 'xgboost',
 'generator': 'optuna-pycaret',
 'file': {'size': '133448', 'sizeHumanized': '133.4 kB'},
 'deployedStatus': 'DONE'}

In [14]:
# 두 번째 순위 높은 모델 선택 (Best Model은 Run후 자동배포되므로..)
model = leaderboard[2] # 여기선 3번째 모델 구함
model

{'id': '87',
 'score': 1.0,
 'trainScore': 0.9619047619047619,
 'validScore': 1.0,
 'testScore': 0.0,
 'estimatorName': 'logistic_regression',
 'generator': 'optuna-pycaret',
 'file': {'size': '1533', 'sizeHumanized': '1.5 kB'},
 'deployedStatus': None}

In [15]:
# 모델 배포
model.deploy()

token expire timie이 28분 이내라 내부적으로 token을 refresh함
{'deployModel': {'deployment': {'id': '5', 'model': {'id': '87', '__typename': 'MlModelBaseType', 'deployedStatus': 'REQUEST'}}}}


In [16]:
# Deployment 정보 구해오기 
deployments = experiment.deployments()
print(f'배포된 모델은 {len(deployments)}개 입니다.')
deployments

배포된 모델은 3개 입니다.


[{'id': '5',
  'name': 'logistic_regression',
  'description': None,
  'status': 'DONE',
  'modelType': 'model',
  'modelPk': 87,
  'allMetricsJson': None,
  'createdAt': '2021-02-10T16:18:36.767495',
  'testScore': None,
  'model': {'id': '87', 'trainScore': 0.9619047619047619, 'validScore': 1.0},
  'file': {'url': 'http://localhost:8000/media/runtime_0003/runtimeprocess_0033/output/pipeline.pkl',
   'size': '1507',
   'sizeHumanized': '1.5 kB',
   'name': 'runtime_0003/runtimeprocess_0033/output/pipeline.pkl'}},
 {'id': '4',
  'name': 'logistic_regression',
  'description': None,
  'status': 'DONE',
  'modelType': 'model',
  'modelPk': 86,
  'allMetricsJson': None,
  'createdAt': '2021-02-10T13:41:22.681704',
  'testScore': None,
  'model': {'id': '86', 'trainScore': 0.9619047619047619, 'validScore': 1.0},
  'file': {'url': 'http://localhost:8000/media/runtime_0003/runtimeprocess_0027/output/pipeline.pkl',
   'size': '2315',
   'sizeHumanized': '2.3 kB',
   'name': 'runtime_0003/runt

In [17]:
deployed_model = deployments[0]
deployed_model

{'id': '5',
 'name': 'logistic_regression',
 'description': None,
 'status': 'DONE',
 'modelType': 'model',
 'modelPk': 87,
 'allMetricsJson': None,
 'createdAt': '2021-02-10T16:18:36.767495',
 'testScore': None,
 'model': {'id': '87', 'trainScore': 0.9619047619047619, 'validScore': 1.0},
 'file': {'url': 'http://localhost:8000/media/runtime_0003/runtimeprocess_0033/output/pipeline.pkl',
  'size': '1507',
  'sizeHumanized': '1.5 kB',
  'name': 'runtime_0003/runtimeprocess_0033/output/pipeline.pkl'}}

In [18]:
# 모델 예측을 위해 Default값인 최빈값을 구함
columns = experiment.column_info()
most_frequent = columns.most_frequent_values()
most_frequent

{'sepal length (cm)': '5.0',
 'sepal width (cm)': '3.0',
 'petal length (cm)': '1.5',
 'petal width (cm)': '0.2'}

In [19]:
# 예측을 위한 input 값 생성 (그냥 최빈값 사용)
input_val = most_frequent
input_val 

{'sepal length (cm)': '5.0',
 'sepal width (cm)': '3.0',
 'petal length (cm)': '1.5',
 'petal width (cm)': '0.2'}

In [20]:
# 예측 
predict_val = deployed_model.predict(input_val)
predict_val

'0.0'